In [2]:
# CELL 1: CONFIGURATION & IMPORTS
import cv2
import mediapipe as mp
import numpy as np
import os
import shutil
import glob
import time
import math

# --- SYSTEM CONFIGURATION ---
# List of poses (Must match folder names in input_images)
POSES_LIST = ["Serve", "DriveForehand", "DriveBackhand"]

# PATH CONFIGURATION
# Since input_images is in the same folder as your code, just use the folder name.
INPUT_ROOT = "input_images"  
ASSETS_ROOT = "assets"      

# MediaPipe Configuration
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Difficulty threshold
SIMILARITY_THRESH = 0.85 

print("Libraries and configuration loaded successfully.")
print(f"Current working directory: {os.getcwd()}")

Libraries and configuration loaded successfully.
Current working directory: d:\pickalball\shawdowing\shadowing_put_frames_like_human-20251218T080101Z-1-001\shadowing_put_frames_like_human


In [3]:
# CELL 2: UTILITY FUNCTIONS

def overlay_smart(background, overlay_img, ghost_meta, user_lms):
    """
    Tự động co giãn (Resize) và căn chỉnh (Align) ảnh bóng ma 
    để khớp với cơ thể người dùng, bất kể độ phân giải ảnh gốc.
    """
    h_bg, w_bg = background.shape[:2] # Kích thước Webcam (VD: 480, 640)
    h_ov, w_ov = overlay_img.shape[:2] # Kích thước ảnh gốc (VD: 3000, 4000)
    
    # 1. Lấy thông số người dùng (Normalized 0.0 -> 1.0)
    # Tính chiều cao thân (Torso Height)
    u_ys = (user_lms[11].y + user_lms[12].y) / 2
    u_yh = (user_lms[23].y + user_lms[24].y) / 2
    u_xh = (user_lms[23].x + user_lms[24].x) / 2 # Tâm hông X
    
    # Chiều cao thân người dùng (tính theo pixel thực tế trên màn hình)
    user_torso_px = abs(u_ys - u_yh) * h_bg
    
    # 2. Lấy thông số Ghost (Normalized 0.0 -> 1.0)
    ghost_torso_ratio = ghost_meta[0] # Tỷ lệ thân trên so với toàn ảnh
    ghost_hip_x = ghost_meta[1]
    ghost_hip_y = ghost_meta[2]
    
    if ghost_torso_ratio == 0 or user_torso_px == 0: return background

    # 3. TÍNH TOÁN KÍCH THƯỚC MỤC TIÊU (QUAN TRỌNG)
    # Công thức: Để thân ghost khớp thân người, chiều cao ảnh ghost mới phải là:
    # New_Height * Ghost_Torso_Ratio = User_Torso_Pixels
    # => New_Height = User_Torso_Pixels / Ghost_Torso_Ratio
    
    target_h = int(user_torso_px / ghost_torso_ratio)
    
    # Tính target_w dựa trên tỷ lệ khung hình gốc (Aspect Ratio)
    aspect_ratio = w_ov / h_ov
    target_w = int(target_h * aspect_ratio)
    
    # Giới hạn an toàn (Safety Clamp): Không cho ảnh quá to (gây lag hoặc tràn hết)
    # Ví dụ: Không cho ảnh to quá 2 lần chiều cao màn hình
    if target_h > h_bg * 2.5:
        target_h = int(h_bg * 2.5)
        target_w = int(target_h * aspect_ratio)

    # 4. Resize ảnh Ghost về kích thước chuẩn vừa tính
    if target_w <= 0 or target_h <= 0: return background
    
    try:
        resized_ghost = cv2.resize(overlay_img, (target_w, target_h))
    except:
        return background

    # 5. Căn chỉnh vị trí (Align Hips)
    # Tọa độ pixel hông người dùng trên Webcam
    u_px_x = int(u_xh * w_bg)
    u_px_y = int(u_yh * h_bg)
    
    # Tọa độ pixel hông Ghost trên ảnh ĐÃ RESIZE
    g_px_x = int(ghost_hip_x * target_w)
    g_px_y = int(ghost_hip_y * target_h)
    
    # Tính điểm đặt (Top-Left) để 2 điểm hông trùng nhau
    top_left_x = u_px_x - g_px_x
    top_left_y = u_px_y - g_px_y
    
    # 6. Cắt cúp và dán ảnh (ROI Processing - Xử lý tràn viền)
    # Tính vùng giao nhau
    y1, y2 = max(0, top_left_y), min(h_bg, top_left_y + target_h)
    x1, x2 = max(0, top_left_x), min(w_bg, top_left_x + target_w)
    
    # Tính vùng tương ứng trên ảnh Ghost
    gy1 = max(0, -top_left_y)
    gy2 = gy1 + (y2 - y1)
    gx1 = max(0, -top_left_x)
    gx2 = gx1 + (x2 - x1)
    
    # Kiểm tra hợp lệ trước khi ghép
    if (y2 > y1) and (x2 > x1) and (gy2 > gy1) and (gx2 > gx1):
        alpha = 0.4 # Độ mờ bóng ma
        
        bg_slice = background[y1:y2, x1:x2]
        ghost_slice = resized_ghost[gy1:gy2, gx1:gx2]
        
        # Tạo mask
        gray_ghost = cv2.cvtColor(ghost_slice, cv2.COLOR_BGR2GRAY)
        _, mask = cv2.threshold(gray_ghost, 5, 255, cv2.THRESH_BINARY)
        
        # Blend
        bg_slice_masked = cv2.bitwise_and(bg_slice, bg_slice, mask=mask)
        ghost_slice_masked = cv2.bitwise_and(ghost_slice, ghost_slice, mask=mask)
        blended = cv2.addWeighted(bg_slice_masked, 1-alpha, ghost_slice_masked, alpha, 0)
        
        # Ghép lại vào background
        inv_mask = cv2.bitwise_not(mask)
        bg_slice_bg = cv2.bitwise_and(bg_slice, bg_slice, mask=inv_mask)
        final_slice = cv2.add(bg_slice_bg, blended)
        
        background[y1:y2, x1:x2] = final_slice

    return background

def calculate_cosine_similarity(user_lms, target_lms_array):
    """
    So sánh vector xương của người dùng và xương mẫu.
    """
    # Các cặp khớp quan trọng (Vai, Khuỷu, Cổ tay, Hông, Đầu gối)
    CONNECTIONS = [
        (11, 13), (13, 15), (12, 14), (14, 16), # Tay
        (11, 23), (12, 24), # Thân
        (23, 25), (24, 26)  # Chân trên
    ]
    
    total_score = 0
    valid_connections = 0
    
    for idx1, idx2 in CONNECTIONS:
        u1 = np.array([user_lms[idx1].x, user_lms[idx1].y])
        u2 = np.array([user_lms[idx2].x, user_lms[idx2].y])
        u_vec = u2 - u1
        
        t1 = target_lms_array[idx1]
        t2 = target_lms_array[idx2]
        t_vec = t2 - t1
        
        norm_u = np.linalg.norm(u_vec)
        norm_t = np.linalg.norm(t_vec)
        
        if norm_u > 0 and norm_t > 0:
            score = np.dot(u_vec, t_vec) / (norm_u * norm_t)
            total_score += score
            valid_connections += 1
            
    if valid_connections == 0: return 0
    return total_score / valid_connections

print("Utility functions updated (Smart Scale Fixed).")

Utility functions updated (Smart Scale Fixed).


In [4]:
# CELL 3: DATA PREPARATION (BUILDER)

def build_assets():
    print("--- START CREATING GHOST DATA (MIRRORED & DEDUPLICATED) ---")
    
    # 1. Clean old assets folder
    if os.path.exists(ASSETS_ROOT): shutil.rmtree(ASSETS_ROOT)
    os.makedirs(ASSETS_ROOT)
    
    # Initialize MediaPipe
    pose_static = mp_pose.Pose(static_image_mode=True, 
                               model_complexity=2, 
                               enable_segmentation=True,
                               min_detection_confidence=0.5)

    total_poses_processed = 0

    # 2. Loop through each Pose
    for pose_name in POSES_LIST:
        input_dir = os.path.join(INPUT_ROOT, pose_name)
        output_dir = os.path.join(ASSETS_ROOT, pose_name)
        
        # DEBUG PRINT
        print(f"Checking folder: {os.path.abspath(input_dir)}")
        
        if not os.path.exists(input_dir):
            print(f"Warning: Folder '{input_dir}' not found. Skipping.")
            continue
            
        os.makedirs(output_dir, exist_ok=True)
        
        # --- TÌM FILE ---
        valid_extensions = ["*.jpg", "*.jpeg", "*.png", "*.JPG", "*.PNG", "*.JPEG"]
        images = []
        for ext in valid_extensions:
            found = glob.glob(os.path.join(input_dir, ext))
            images.extend(found)
        
        # --- SỬA LỖI DUPLICATE TẠI ĐÂY ---
        # Lọc bỏ các file trùng lặp do Windows không phân biệt hoa thường
        images = list(set(images))
        # Sắp xếp lại để đảm bảo thứ tự 1, 2, 3, 4
        images = sorted(images)
        # ---------------------------------
        
        if len(images) != 4:
            print(f"Error: Folder '{pose_name}' needs exactly 4 images. Found {len(images)}.")
            print(f"List: {images}")
            continue
            
        print(f"Processing: {pose_name.upper()} ({len(images)} images)...")
        
        # Process each image
        for i, img_path in enumerate(images):
            frame = cv2.imread(img_path)
            if frame is None: 
                print(f"   -> Cannot read file: {img_path}")
                continue
            
            # --- LẬT ẢNH ĐẦU VÀO (MIRROR) ---
            frame = cv2.flip(frame, 1) 
            # --------------------------------
            
            # MediaPipe Process
            img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose_static.process(img_rgb)
            
            if results.pose_landmarks and results.segmentation_mask is not None:
                # A. Create Ghost Image
                mask = (results.segmentation_mask > 0.5).astype(np.uint8) * 255
                ghost = np.zeros_like(frame)
                ghost[:] = (0, 255, 0) # Green color
                ghost_masked = cv2.bitwise_and(ghost, ghost, mask=mask)
                contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                cv2.drawContours(ghost_masked, contours, -1, (255, 255, 255), 2)
                cv2.imwrite(f"{output_dir}/ghost_{i}.png", ghost_masked)
                
                # B. Calculate Metadata
                lms = results.pose_landmarks.landmark
                ys = (lms[11].y + lms[12].y) / 2
                yh = (lms[23].y + lms[24].y) / 2
                torso_h = abs(ys - yh)
                xh = (lms[23].x + lms[24].x) / 2
                yh_c = (lms[23].y + lms[24].y) / 2
                np.save(f"{output_dir}/meta_{i}.npy", [torso_h, xh, yh_c])
                
                # C. Save Target Skeleton
                landmarks_xy = []
                for lm in lms:
                    landmarks_xy.append([lm.x, lm.y])
                np.save(f"{output_dir}/target_{i}.npy", np.array(landmarks_xy))
                
            else:
                print(f"   -> No person found in image: {os.path.basename(img_path)}")
        
        total_poses_processed += 1
        print(f"   -> Done {pose_name}")

    pose_static.close()
    
    if total_poses_processed > 0:
        print("\nPROCESSING COMPLETED! Data is ready in 'assets' folder.")
        print("Please run Cell 4 to start training.")
    else:
        print("\nNOTHING PROCESSED. Please check the paths printed above.")

# Run build function
build_assets()

--- START CREATING GHOST DATA (MIRRORED & DEDUPLICATED) ---
Checking folder: d:\pickalball\shawdowing\shadowing_put_frames_like_human-20251218T080101Z-1-001\shadowing_put_frames_like_human\input_images\Serve
Processing: SERVE (4 images)...
   -> Done Serve
Checking folder: d:\pickalball\shawdowing\shadowing_put_frames_like_human-20251218T080101Z-1-001\shadowing_put_frames_like_human\input_images\DriveForehand
Processing: DRIVEFOREHAND (4 images)...
   -> Done DriveForehand
Checking folder: d:\pickalball\shawdowing\shadowing_put_frames_like_human-20251218T080101Z-1-001\shadowing_put_frames_like_human\input_images\DriveBackhand
Processing: DRIVEBACKHAND (4 images)...
   -> Done DriveBackhand

PROCESSING COMPLETED! Data is ready in 'assets' folder.
Please run Cell 4 to start training.


In [6]:
# CELL 4: ADAPTIVE GHOST TRAINER (RUNNER)

def run_ghost_trainer():
    # Check data
    if not os.path.exists(ASSETS_ROOT):
        print("Error: No data found. Please run Cell 3 first!")
        return

    # Initialize Webcam & MediaPipe
    cap = cv2.VideoCapture(0) # Camera index 0
    pose_tracker = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
    
    # State variables
    current_pose_idx = 0  # 0: Serve, 1: Forehand, 2: Backhand
    current_stage = 0     # 0 -> 3 (4 stages)
    
    # Logic variables
    last_match_time = 0
    match_duration = 0 
    REQUIRED_HOLD_TIME = 0.5 # Giữ 0.5s để qua màn
    
    # --- THÊM BIẾN COOLDOWN ---
    cooldown_start = 0
    COOLDOWN_TIME = 2.0 # Nghỉ 2 giây giữa các bước
    in_cooldown = False
    # --------------------------
    
    print("\n--- TRAINING STARTED ---")
    print("Shortcuts: [n] Next Pose | [r] Retry | [q] Quit")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        
        # 1. Prepare Image
        frame = cv2.flip(frame, 1) # Mirror flip
        h, w = frame.shape[:2]
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # 2. Detect User Skeleton
        results = pose_tracker.process(img_rgb)
        
        # Determine current exercise
        pose_name = POSES_LIST[current_pose_idx]
        asset_path = os.path.join(ASSETS_ROOT, pose_name)
        
        # UI Text Font
        font = cv2.FONT_HERSHEY_SIMPLEX

        # XỬ LÝ COOLDOWN (Nếu đang nghỉ thì không chấm điểm)
        if in_cooldown:
            elapsed = time.time() - cooldown_start
            remaining = int(COOLDOWN_TIME - elapsed) + 1
            
            # Hiển thị thông báo nghỉ
            cv2.putText(frame, f"GET READY FOR STEP {current_stage + 1}...", (50, h//2), font, 1.5, (0, 255, 255), 3)
            cv2.putText(frame, f"Starts in: {remaining}s", (50, h//2 + 50), font, 1, (255, 255, 255), 2)
            
            # Hiện mờ cái bóng tiếp theo để người dùng chuẩn bị
            next_ghost_path = f"{asset_path}/ghost_{current_stage}.png"
            if os.path.exists(next_ghost_path):
                 # Code load ảnh ở đây chỉ để hiển thị visual (không tính điểm)
                 temp_ghost = cv2.imread(next_ghost_path)
                 temp_meta = np.load(f"{asset_path}/meta_{current_stage}.npy")
                 if results.pose_landmarks:
                     frame = overlay_smart(frame, temp_ghost, temp_meta, results.pose_landmarks.landmark)

            if elapsed > COOLDOWN_TIME:
                in_cooldown = False # Hết giờ nghỉ, bắt đầu tập
                last_match_time = time.time() # Reset timer chấm điểm
            
            cv2.imshow("Pickleball Ghost Trainer", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'): break
            continue # Bỏ qua phần dưới, quay lại đầu vòng lặp
            
        # -----------------------------------------------------------

        # If all 4 stages completed
        if current_stage >= 4:
            cv2.putText(frame, f"{pose_name.upper()} COMPLETE!", (50, h//2), font, 2, (0, 255, 0), 4)
            cv2.putText(frame, "Press 'r' to replay or 'n' for next", (50, h//2 + 60), font, 0.8, (255, 255, 255), 2)
        else:
            # Load Data for current Stage
            ghost_img_path = f"{asset_path}/ghost_{current_stage}.png"
            meta_path = f"{asset_path}/meta_{current_stage}.npy"
            target_path = f"{asset_path}/target_{current_stage}.npy"
            
            if os.path.exists(ghost_img_path):
                ghost_img = cv2.imread(ghost_img_path)
                ghost_meta = np.load(meta_path)
                target_lms = np.load(target_path)
                
                sim_score = 0
                
                # 3. Logic & Visual Processing
                if results.pose_landmarks:
                    user_lms = results.pose_landmarks.landmark
                    
                    # A. Smart Overlay
                    frame = overlay_smart(frame, ghost_img, ghost_meta, user_lms)
                    
                    # B. Scoring
                    sim_score = calculate_cosine_similarity(user_lms, target_lms)
                    
                    # C. Check Result
                    if sim_score > SIMILARITY_THRESH:
                        color = (0, 255, 0) # Green
                        match_duration = time.time() - last_match_time
                        
                        # Draw Progress Bar
                        ratio = match_duration / REQUIRED_HOLD_TIME
                        current_bar_w = min(200, int(ratio * 200))
                        
                        pt1 = (50, 100)
                        pt2_x = int(50 + current_bar_w)
                        pt2_y = 120
                        pt2 = (pt2_x, pt2_y)
                        
                        cv2.rectangle(frame, pt1, pt2, (0, 255, 0), -1)
                        cv2.rectangle(frame, (50, 100), (250, 120), (255, 255, 255), 2)
                        
                        if match_duration > REQUIRED_HOLD_TIME:
                            # --- QUA MÀN THÀNH CÔNG ---
                            cv2.putText(frame, "PERFECT!", (50, h//2), font, 2, (0, 255, 0), 4)
                            cv2.imshow("Pickleball Ghost Trainer", frame)
                            cv2.waitKey(500) # Dừng hình 0.5s cho đẹp
                            
                            current_stage += 1
                            match_duration = 0
                            
                            # Kích hoạt Cooldown trước khi sang bước tiếp theo
                            if current_stage < 4:
                                in_cooldown = True
                                cooldown_start = time.time()
                            # --------------------------
                    else:
                        color = (0, 0, 255) # Red
                        last_match_time = time.time() 
                        match_duration = 0

                    # Display Score
                    cv2.putText(frame, f"SCORE: {int(sim_score*100)}%", (50, 80), font, 1.2, color, 3)
                else:
                    cv2.putText(frame, "Please stand in frame", (50, 80), font, 1, (0, 255, 255), 2)
            else:
                cv2.putText(frame, "Missing Data File!", (50, 50), font, 1, (0,0,255), 2)

            # UI Info
            cv2.putText(frame, f"POSE: {pose_name.upper()}", (20, 40), font, 1, (255, 255, 0), 2)
            cv2.putText(frame, f"Step: {current_stage + 1}/4", (w - 200, 40), font, 1, (255, 255, 255), 2)

        cv2.imshow("Pickleball Ghost Trainer", frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'): break
        if key == ord('r'): 
            current_stage = 0
            in_cooldown = False
            print("Reset stage.")
        if key == ord('n'):
            current_pose_idx = (current_pose_idx + 1) % len(POSES_LIST)
            current_stage = 0
            in_cooldown = False
            print(f"Switched to: {POSES_LIST[current_pose_idx]}")

    cap.release()
    pose_tracker.close()
    cv2.destroyAllWindows()

# Run program
run_ghost_trainer()


--- TRAINING STARTED ---
Shortcuts: [n] Next Pose | [r] Retry | [q] Quit
